# Graph Node Classification with ThirdAI's Universal Deep Transformer
This notebook shows how to build a node prediction model with ThirdAI's Universal Deep Transformer (UDT) model. In this demo, we will train and evaluate the model on the YelpCHI dataset, but you can easily replace this with your own dataset.

You can immediately run a version of this notebook in your browser on Google Colab at the following link:

https://githubtocolab.com/ThirdAILabs/Demos/blob/main/GraphNodeClassification.ipynb

This notebook uses an activation key that will only work with this demo. If you want to try us out on your own dataset, you can obtain a free trial license at the following link: https://www.thirdai.com/try-bolt/

In [ ]:
!pip3 install thirdai --upgrade
!pip3 install scikit-learn

# This activates the ThirdAI package with a key that is only good for this demo
import thirdai

thirdai.licensing.activate("MWHE-FVNH-3XN3-KWXY-TFRA-X3VJ-U7FT-RLVR")

# Dataset Download
We will use the demos module in the thirdai package to download the YelpCHI dataset. You can replace this step and the next step with a download method and a UDT initialization step that is specific to your dataset.

In [ ]:
from thirdai.demos import download_yelp_chi_dataset

train_data_path, _, inference_samples, _ = download_yelp_chi_dataset()

# UDT Initialization
We can now create a UDT model by passing in the types of each column in the dataset and the target column we want to be able to predict. The input format for training UDT on a graph dataset is a typical CSV, where every row represents a single node in the graph. The bolt.types.node_id() column should be a unique integer identifying the node, while the bolt.types.neighbors() column should be a space separated list of node ids corresponding to the node's neighbors. For YelpChi, each node additionally has 32 numerical features (a "feature vector"), as well as a target column we wish to predict.

In [ ]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "node_id": bolt.types.node_id(),
        **{f"col_{i}": bolt.types.numerical([0, 1]) for i in range(32)},
        "target": bolt.types.categorical(),
        "neighbors": bolt.types.neighbors(),
    },
    target="target",
    n_target_classes=2,
    integer_target=True,
    # Optional: you can remove the next line to increase AUC to ~0.93 at the
    # cost of increasing training time
    options={"contextual_columns": False},
)

# Indexing
Graph models with UDT require an additional *indexing* step before training. Since nodes in the train set have neighbors in the eval set, we must tell the model about the node features of nodes in the eval set before training. This effectively gives the UDT model complete knowledge of the graph structure and features before training. The eval set has all labels set to 0, so this is not leaking testing information.

In [ ]:
model.index_nodes(eval_data_path)

# Training
We can now train our UDT graph model! Feel free to customize the number of epochs and the learning rate; we have chosen values that give good convergence.

In [ ]:
model.train(
    train_data_path, learning_rate=0.001, epochs=15, metrics=["categorical_accuracy"]
)

# Evaluation
We can evaluate the performance of UDT by calculating the Area Under the Receiver Operating Characteristic Curve (ROC AUC). UDT achieves about 91% with the default configurations in this notebook, and as noted above we achieve 93-94% with longer training time (the previous SOTA was [90%](https://paperswithcode.com/sota/node-classification-on-yelpchi)). Here, we show the predict API, where we pass in samples without labels.

In [ ]:
import numpy as np
from sklearn import metrics

ground_truth = [inference_sample[1] for inference_sample in inference_samples]

predictions = []
ground_truths = []
for sample, ground_truth in inference_samples:
    prediction = model.predict(sample)
    predictions.append(prediction)
    ground_truths.append(ground_truth)
predictions = np.array(predictions)

auc = metrics.roc_auc_score(ground_truths, predictions[:, 1])

print("Test AUC:", auc)